# <center> Matching of the SDG </center>


To classify each ecuadorian governmental project of investment, we design a matching process between each project as one (or none) of the UN SDG. This required:

- To get the name of each project of investment of the Ecuadorian Government (See the PAI database).
- To get the specific SDG (target) of each SDG

 We are using as training data, the information of the ecuadorian government flow of investment from 2017 to 2022, which has flows asociated to a target. The files required are *ALINEACION_2017-2022_.xlsx* (given by: Director Planificación,Seguimiento y Evaluación Institucional), and *pnd_ods.xlsx* (Made manually and ussing information from http://oportunidades.planificacion.gob.ec/Plan2125/ for data between 2021-2025, and https://bicloud.planificacion.gob.ec/pentaho/api/repos/:public:SNIObjetivos:Objetivos.wcdf/generatedContent?userid=pentahopublic&password=pentaho for data between 2017-2021). 

 The sdg and target are given by the undp we site in spanish: https://www.un.org/sustainabledevelopment/es/objetivos-de-desarrollo-sostenible/. They were manually saved on the file *targets.csv*

## Cleaning Up

The procces classify a text input as one or more than one target (which implies that is also classifyed as sdg). For that, it is required a training procces. First, we have a set of investment flow in the file *ALINEACION_2017-2022_.xlsx*, and they are also classyfied as an objective from the ecuadorian governmental plan of development (Plan Nacional de Desarollo de la Secretaria de Planificación del Ecuador). The file *pnd_ods.xlsx* let us knwo which target/sdg is matched by the ecuadorian objetive.

In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode

# alineacion data frame

alineacion_2017 = pd.read_excel('raw data/ALINEACION_2017-2022_.xlsx', sheet_name='2017')
alineacion_2018 = pd.read_excel('raw data/ALINEACION_2017-2022_.xlsx', sheet_name='2018')
alineacion_2019 = pd.read_excel('raw data/ALINEACION_2017-2022_.xlsx', sheet_name='2019')
alineacion_2020 = pd.read_excel('raw data/ALINEACION_2017-2022_.xlsx', sheet_name='2020')
alineacion_2021 = pd.read_excel('raw data/ALINEACION_2017-2022_.xlsx', sheet_name='2021')
alineacion_2022 = pd.read_excel('raw data/ALINEACION_2017-2022_.xlsx', sheet_name='2022')

alineacion = pd.concat([alineacion_2017, alineacion_2018, alineacion_2019, alineacion_2020, alineacion_2021, alineacion_2022])

alineacion

,ANIO_ALINEACION,RUC_ENTIDAD,CODIGO_MEF,ENTIDAD_NOMBRE,CUP_PROYECTO,NOMBRE_PROYECTO,MONTO_TOTAL,PRY_FECHA_INICIO,PRY_FECHA_FIN,OBJETIVO_PND,POLITICA_PND,META_PND,OBJETIVO_ESTRATEGICO,CODIGO_PROGRAMA_PRESUPUETARIO,NOMBRE_PROGRAMA_PRESUPUESTARIO,
0,2017,1768168210001,47,AGENCIA DE REGULACIÓN Y CONTROL DE LA BIOSEGUR...,040470000.0000.376147,"CONSOLIDACIÓN DEL SISTEMA DE PREVENCIÓN, CONTR...",10593834.00,01/01/2014,31/12/2017,OBJETIVO 07: GARANTIZAR LOS DERECHOS DE LA NAT...,GARANTIZAR LA BIOSEGURIDAD PRECAUTELANDO LA SA...,"NO DISMINUIR DE 0,35 HECTÁREAS PER CÁPITA LA B...",REDUCIR EL RIESGO DE INTRODUCCIÓN Y DISPERSION...,55,REGULACION PREVENCION Y CONTROL PARA LA BIOSEG...,
1,2017,1768158680001,416,AGENCIA DE REGULACION Y CONTROL HIDROCARBURIFE...,144160000.0000.375835,EQUIPAMIENTO DEL LABORATORIO DE HIDROCARBUROS ...,1900000.00,05/01/2015,09/12/2016,OBJETIVO 01: CONSOLIDAR EL ESTADO DEMOCRÁTICO ...,MEJORAR LA FACULTAD REGULADORA Y DE CONTROL DE...,AUMENTAR EL ÍNDICE DE CAPACIDAD INSTITUCIONAL ...,INCREMENTAR EL CONTROL Y LA FISCALIZACIÓN DE L...,55,REGULACION Y CONTROL HIDROCARBURIFERO,
2,2017,1768155150001,426,AGENCIA DE REGULACION Y CONTROL MINERO-ARCOM,144260000.0000.372784,"PROYECTO DE SEGUIMIENTO, CONTROL Y EVALUACIÓN ...",4347087.75,01/01/2012,31/12/2017,OBJETIVO 11: ASEGURAR LA GESTIÓN SOBERANA Y EF...,INDUSTRIALIZAR LA ACTIVIDAD MINERA COMO EJE DE...,IDENTIFICAR LA DISPONIBILIDAD DE OCURRENCIAS D...,INCREMENTAR LA REGULACIÓN Y CONTROL DE LAS ACT...,55,REGULACION Y CONTROL MINERO,
3,2017,1768155150001,426,AGENCIA DE REGULACION Y CONTROL MINERO-ARCOM,144260000.0000.375945,PROYECTO IMPLEMENTACIÓN DEL SISTEMA DE AUDITOR...,26195688.00,01/01/2014,31/12/2017,OBJETIVO 11: ASEGURAR LA GESTIÓN SOBERANA Y EF...,INDUSTRIALIZAR LA ACTIVIDAD MINERA COMO EJE DE...,IDENTIFICAR LA DISPONIBILIDAD DE OCURRENCIAS D...,INCREMENTAR LA REGULACIÓN Y CONTROL DE LAS ACT...,55,REGULACION Y CONTROL MINERO,
4,2017,1768155150001,426,AGENCIA DE REGULACION Y CONTROL MINERO-ARCOM,144260000.0000.375946,DETERMINACIÓN DE LA ZONA DE SEGURIDAD Y AMPLIA...,3835103.83,01/01/2014,30/12/2016,OBJETIVO 11: ASEGURAR LA GESTIÓN SOBERANA Y EF...,INDUSTRIALIZAR LA ACTIVIDAD MINERA COMO EJE DE...,IDENTIFICAR LA DISPONIBILIDAD DE OCURRENCIAS D...,REDUCIR LA MINERÍA ILEGAL E INFORMAL EN TODO E...,56,DISMINUCION DE LA MINERIA ILEGAL E INFORMAL,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,2022,560001270001,179,UNIVERSIDAD TÉCNICA DE COTOPAXI,91790000.0000.382662,ENERGÍA Y TECNOLOGÍA AL SERVICIO DE LA SOCIEDAD,82496.68,02/10/2017,31/12/2021,E2.O7. POTENCIAR LAS CAPACIDADES DE LA CIUDADA...,E2.O7.P4. FORTALECER EL SISTEMA DE EDUCACIÓN S...,E2.O7.P4.M1. INCREMENTAR LOS ARTÍCULOS PUBLICA...,3. GARANTIZAR EL USO SOCIAL DEL CONOCIMIENTO C...,84,GESTIÓN DE LA VINCULACIÓN CON LA COLECTIVIDAD,
704,2022,1360002090001,173,UNIVERSIDAD TÉCNICA DE MANABÍ,91730000.0000.383309,CONSTRUCCION DE EDIFICIO III PARA AULAS OFICIN...,3369806.47,01/09/2018,31/08/2019,E2.O7. POTENCIAR LAS CAPACIDADES DE LA CIUDADA...,E2.O7.P4. FORTALECER EL SISTEMA DE EDUCACIÓN S...,E2.O7.P4.M4. INCREMENTAR EL NÚMERO DE INVESTIG...,"INCREMENTAR LA COBERTURA, MATRÍCULA, TASA DE R...",82,FORMACION Y GESTION ACADEMICA,
705,2022,1360002090001,173,UNIVERSIDAD TÉCNICA DE MANABÍ,91730000.0000.383309,CONSTRUCCION DE EDIFICIO III PARA AULAS OFICIN...,3369806.47,01/09/2018,31/08/2019,E2.O7. POTENCIAR LAS CAPACIDADES DE LA CIUDADA...,E2.O7.P4. FORTALECER EL SISTEMA DE EDUCACIÓN S...,E2.O7.P1.M2. INCREMENTAR LA TASA BRUTA DE MATR...,"INCREMENTAR LA COBERTURA, MATRÍCULA, TASA DE R...",82,FORMACION Y GESTION ACADEMICA,
706,2022,1260001380001,172,UNIVERSIDAD TÉCNICA ESTATAL DE QUEVEDO,91720000.1069.4645,ADQUISICION DE ACTIVOS Y DE CONSTRUCCIONES,14053775.62,01/01/2016,31/12/2020,E2.O7. POTENCIAR LAS CAPACIDADES DE LA CIUDADA...,E2.O7.P4. FORTALECER EL SISTEMA DE EDUCACIÓN S...,E2.O7.P4.M2. INCREMENTAR LA TASA BRUTA DE MATR...,INCREMENTAR LA CALIDAD ACADÉMICA DE LAS CARRER...,82,FORMACION Y GESTION 

 We know with this file the informatin of the governental objectives, to see to which target and sdg we read the file *pnd_ods.xlsx*

In [2]:
# pnd and ods for (spanish) "Plan Nacional de Desarrollo" and "Objetivo de Desarrollo Sostenible" (National Plan of Development and Sustainable Development Goal)

pnd_ods = pd.read_csv('raw data/pnd_ods.csv')
pnd_ods

,META,OBJETIVO,DESCRIPTION,sdg,initial,end
0,1.1.1.,"Objetivo 1.- Incrementar y fomentar, de manera...",Meta 1.1.1. Incrementar la tasa de empleo adec...,8.5,2021,2025
1,1.1.2.,"Objetivo 1.- Incrementar y fomentar, de manera...",Meta 1.1.2. Reducir la tasa de desempleo juven...,8.6,2021,2025
2,1.1.3.,"Objetivo 1.- Incrementar y fomentar, de manera...",Meta 1.1.3. Incrementar el porcentaje de perso...,NaN,2021,2025
3,1.1.4.,"Objetivo 1.- Incrementar y fomentar, de manera...",Meta 1.1.4. Aumentar el número de personas con...,8.5,2021,2025
4,1.1.5.,"Objetivo 1.- Incrementar y fomentar, de manera...",Meta 1.1.5. Incrementar para el 2025 la tasa a...,NaN,2021,2025
...,...,...,...,...,...,...
299,9.5,"objetivo 9: garantizar la soberanía y la paz, ...",9.5 incrementar el ingreso de divisas por conc...,NaN,2017,2021
300,9.6,"objetivo 9: garantizar la soberanía y la paz, ...",9.6 incrementar el cumplimiento de compromisos...,NaN,2017,2021
301,9.8,"objetivo 9: garantizar la soberanía y la paz, ...",9.8 mejorar el resultado de la balanza comerci...,NaN,2017,2021
302,9.9,"objetivo 9: garantizar la soberanía y la paz, ...",9.9 incrementar la población con empleo en las...,NaN,2017,2021


The last required df is the target df.

The variables meaning are:
    
- **META**: specific objective index from the ecuadorian pnd. It is different in the period 2017-2021 from 20221-2025 (For the change of the president.)
- **description**: Description of the general objective
- **DESCRIPTION**: Specific objective by the PND
- **sdg**: An index from the target and sdg. The UNDP website uses the goal and numbering to the index the targets. Sometimes, the website uses letters instead of numbers. We changed the index and it is available in the file *targets.csv*. So, the META 1.1.1. with sdg 8.5, mean sdg 8 with target 5. The space between numbers means that the objective fill more than one target of the UNDP (META 9.5 with sdg "6 14 15" means that the ecuadorian 9.5 fills the sdg 6, 14, and 15).  
- **initial**: The initial year of the presidential period.
- **end**: The last year of the presidential period. The observations in *alineacion* df with year the 2021, correspond to the period 2017 to 2021 in *pnd_ods* df. 

In [11]:
undp = pd.read_csv('raw data/targets.csv')
undp

,target,sdg,sdg_information,target_information
0,G1T1,G1,Poner fin a la pobreza en todas sus formas en ...,"Para 2030, erradicar la pobreza extrema para t..."
1,G1T2,G1,Poner fin a la pobreza en todas sus formas en ...,"Para 2030, reducir al menos a la mitad la prop..."
2,G1T3,G1,Poner fin a la pobreza en todas sus formas en ...,Poner en práctica a nivel nacional sistemas y ...
3,G1T4,G1,Poner fin a la pobreza en todas sus formas en ...,"Para 2030, garantizar que todos los hombres y ..."
4,G1T5,G1,Poner fin a la pobreza en todas sus formas en ...,"Para 2030, fomentar la resiliencia de los pobr..."
...,...,...,...,...
163,G16T14,G16,"Promover sociedades justas, pacíficas e inclus...",Revitalizar la Alianza Mundial para el Desarro...
164,G16T15,G16,"Promover sociedades justas, pacíficas e inclus...",Revitalizar la Alianza Mundial para el Desarro...
165,G16T16,G16,"Promover sociedades justas, pacíficas e inclus...",Revitalizar la Alianza Mundial para el Desarro...
166,G16T17,G16,"Promover sociedades justas, pacíficas e inclus...",Revitalizar la Alianza Mundial para el Desarro...


- **id**: Is the observation index. The G represent the UNDP SDG goal. The T the specific target of the goal.
- **goal**: An indicator por the sdg
- **goal_information**: Text information of the sdg
- **target_information**: Text information of the sdg target

In [5]:
# Select text

objetivo = pnd_ods['OBJETIVO'].to_list() # Text information part 1
description = pnd_ods['DESCRIPTION'].to_list() # Text information part 2
pnd_sdg_raw = pnd_ods['sdg'].to_list() # sdg.target

pnd_sdg = ['sdg_non_classified']*len(pnd_sdg_raw) # sdg per objetive
pnd_target = ['target_non_classified']*len(pnd_sdg_raw) # target per objetive

for n in range(len(objetivo)):
    
    # Some important changes to the text information
    
    objetivo[n] = objetivo[n].split(' ')
    
    if (objetivo[n][0] == 'Objetivo') and any(char.isdigit() for char in objetivo[n][1]):
        
        objetivo[n] = objetivo[n][2:]
        
    objetivo[n] = ' '.join(objetivo[n])
        
    description[n] = description[n].split(' ')
    
    if ('Meta' == description[n][0]) and any(char.isdigit() for char in description[n][1]):
        
        description[n] = description[n][2:]
        
    description[n] = ' '.join(description[n])
    
    # get sdg and target
    
    pnd_sdg_raw[n] = str(pnd_sdg_raw[n])
    
    if pnd_sdg_raw[n] != 'nan':
    
        pnd_sdg_raw[n] = pnd_sdg_raw[n].split(' ')    
        
        
        g = ['GT']*len(pnd_sdg_raw[n])
        k = ['G']*len(pnd_sdg_raw[n])
        
        for m in range(len(pnd_sdg_raw[n])):
            
            r = 0 # if one, the value wont be appended to the sdg
            
            for c in range(len(pnd_sdg_raw[n][m])):
                
                if pnd_sdg_raw[n][m][c] == '.':
                    r = 1
                
                else:
                
                    if r != 1:
                    
                        k[m]+=pnd_sdg_raw[n][m][c]
                        g[m] = g[m].replace('T', pnd_sdg_raw[n][m][c]+'T')
                        
                    else:
                        
                        g[m]+=pnd_sdg_raw[n][m][c]
            
        k = list(set(k))
        pnd_sdg[n] = ' '.join(k)
        
        pnd_target[n] = ' '.join(g)
    
    
pnd_text = [i+'. '+j for (i, j) in zip(objetivo, description)]

# ## clean function: Will be useful for future df

def clean_text(text_list):
    
    text_list = [i.lower() for i in text_list]
    text_list = [i.replace('-','') for i in text_list]
    
    text_list = [x.replace(',', '') for x in text_list]
    text_list = [x.replace('  ', ' ') for x in text_list]
    text_list = [x.replace('.', '') for x in text_list]
    text_list = [x.replace('(', '') for x in text_list]
    text_list = [x.replace(')', '') for x in text_list]
    text_list = [x.replace('%', '') for x in text_list]
    text_list = [x.replace(':', '') for x in text_list]
    text_list = [x.replace('?', '') for x in text_list]
    text_list = [unidecode(i) for i in text_list]

    for m in range(len(text_list)):
    
        text_list[m] = text_list[m].split(' ')
        
        if (text_list[m][0] == 'objetivo') and any(char.isdigit() for char in text_list[m][1]):
            
            text_list[m] = text_list[m][2:]
        
        text_list[m] = [j for j in text_list[m] if (any(char.isdigit() for char in j) == False)]
            
        k = []    
        
        for w in text_list[m]:
            
            w = ''.join([i for i in w if not i.isdigit()])
            
            if w != '':
            
               k.append(w)
               
        text_list[m] = ' '.join(k)
        
    return text_list

pnd_text = clean_text(pnd_text)

pnd_df = pd.DataFrame({
    'target': pnd_target,
    'sdg': pnd_sdg,
    'text': pnd_text
})
pnd_df

,target,sdg,text
0,G8T5,G8,incrementar y fomentar de manera inclusiva las...
1,G8T6,G8,incrementar y fomentar de manera inclusiva las...
2,target_non_classified,sdg_non_classified,incrementar y fomentar de manera inclusiva las...
3,G8T5,G8,incrementar y fomentar de manera inclusiva las...
4,target_non_classified,sdg_non_classified,incrementar y fomentar de manera inclusiva las...
...,...,...,...
299,target_non_classified,sdg_non_classified,garantizar la soberania y la paz y posicionar ...
300,target_non_classified,sdg_non_classified,garantizar la soberania y la paz y posicionar ...
301,target_non_classified,sdg_non_classified,garantizar la soberania y la paz y posicionar ...
302,target_non_classified,sdg_non_classified,garantizar la soberania y la paz y posicionar ...


 Geting more data from how the government is classifying their projects. The varaible *META_PND* must be equal to *DESCRIPTION* variable from *pnd_ods* df.

In [6]:
# Get text

al_entidad = alineacion['ENTIDAD_NOMBRE'].to_list()
al_nombre = alineacion['NOMBRE_PROYECTO'].to_list()
al_programa = alineacion['NOMBRE_PROGRAMA_PRESUPUESTARIO'].to_list()

al_text = [i+'. '+j+'. '+k for (i,j,k) in zip(al_entidad, al_nombre, al_programa)]
al_text = clean_text(al_text)

# Conection with previous dataframe

al_conection = alineacion['META_PND'].to_list()
al_conection = clean_text(al_conection)
al_sdg = ['sdg_non_classified']*len(al_conection)
al_target = ['target_non_classified']*len(al_conection)


for s in range(len(al_conection)):
    
    for p in range(len(pnd_text)):
        
        if al_conection[s] in pnd_text[p]:
            
            
        
            if pnd_target[p] != 'target_non_classified':
                
                if al_target[s] == 'target_non_classified':
                    al_target[s] = pnd_target[p]
                    al_sdg[s] = pnd_sdg[p]
                    
                else:
                    al_target[s] = al_target[s]+' '+pnd_target[p]
                    al_sdg[s] = pnd_sdg[p]+' '+pnd_sdg[p]
                    
            elif pnd_sdg[p] != 'sdg_non_classified':
                
                if pnd_sdg[s] == 'sdg_non_classified':
                
                    al_sdg[s] = pnd_sdg[p]
                    
                else:
                    al_sdg[s] = pnd_sdg[p]+' '+pnd_sdg[p]
                    
# Create an index to make eassier to call observations 

al_df_index = [f'BASE-O-{i}' for i in range(1,len(al_target)+1)]           

al_df = pd.DataFrame({
    'target': al_target,
    'sdg': al_sdg,
    'text': al_text
}, index=al_df_index)
al_df

,target,sdg,text
BASE-O-1,target_non_classified,sdg_non_classified,agencia de regulacion y control de la biosegur...
BASE-O-2,target_non_classified,sdg_non_classified,agencia de regulacion y control hidrocarburife...
BASE-O-3,target_non_classified,sdg_non_classified,agencia de regulacion y control mineroarcom pr...
BASE-O-4,target_non_classified,sdg_non_classified,agencia de regulacion y control mineroarcom pr...
BASE-O-5,target_non_classified,sdg_non_classified,agencia de regulacion y control mineroarcom de...
...,...,...,...
BASE-O-6411,G9T5,G9,universidad tecnica de cotopaxi energia y tecn...
BASE-O-6412,G9T5,G9,universidad tecnica de manabi construccion de ...
BASE-O-6413,target_non_classified,sdg_non_classified,universidad tecnica de manabi construccion de ...
BASE-O-6414,G4T3 G4T5 G4T9,G4,universidad tecnica estatal de quevedo adquisi...


 The next step is to transform the uniques values form target and sdg columns into variables. In other words, a column per each unique variable. Every one of these new variables will be valued by 1 if the value was classified as this variable, 0 otherwise. The column *target_non_classified* implies that the observation must not be classified as any target, but could be classified as a sdg. The column *sdg_non_classified* implies that the observation must not be classified as either a sdg or a target. An observation classified as a target, also must be classified as the corresponding target sdg. An observation can be classified as a sdg but non classified as any of their sdg targets.

In [7]:
# The length of the matrix columns must be equal to length of non_missing index

N = len(al_df)

def gen_uniques(text_list):

    text_list = ' '.join(text_list)
    text_list = text_list.split(' ')
    text_list = list(set(text_list))

    return text_list

# The words will be the input of the model

complete_text = al_df['text'].to_list()
complete_text_uniques = gen_uniques(complete_text)


# The output will be the sdg and the targets

sdg = al_df['sdg'].to_list()
sdg_uniques = gen_uniques(sdg)

targets = al_df['target'].to_list()
targets_uniques = gen_uniques(targets)

# Creating value matrix:

training_dictionary = {}

for v in targets_uniques+sdg_uniques:
    
    new_vector = [0]*N
    
    for n in range(N):
        
        if v in sdg[n]:
            new_vector[n] = 1
            
        if v in targets[n]:
            
            new_vector[n] = 1
                
        if v in complete_text[n]:
            
            new_vector[n] = 1
            
    
    training_dictionary[v] = new_vector
    
train = pd.concat([al_df['text'], pd.DataFrame(training_dictionary, index=al_df_index)], axis="columns")
train.index.names = ['id']
train.to_csv('train/train.csv')


print('Train data rows: ', len(train.index))
print('Train data columns: ', len(train.columns))

train

Train data rows:  6415
Train data columns:  42


,text,G13T,target_non_classified,G4T1,G4T9,G1T1,G1T3,G8T7,G1T2,G9T,...,G2,sdg_non_classified,G12,G16,G3,G8,G14,G4,G13,G9
id,,,,,,,,,,,,,,,,,,,,,
BASE-O-1,agencia de regulacion y control de la biosegur...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
BASE-O-2,agencia de regulacion y control hidrocarburife...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
BASE-O-3,agencia de regulacion y control mineroarcom pr...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
BASE-O-4,agencia de regulacion y control mineroarcom pr...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
BASE-O-5,agencia de regulacion y control mineroarcom de...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BASE-O-6411,universidad tecnica de cotopaxi energia y tecn...,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
BASE-O-6412,universidad tecnica de manabi construccion de ...,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
BASE-O-6413,universidad tecnica de manabi construccion de ...,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


 We also need two more data sets to train and classify the model. A dataset to classify all targets types, and other to classify all sdg types

In [36]:
undp['text'] = undp['sdg_information'] + undp['target_information']
targets = undp[['target', 'text']]
targets.set_index('target', inplace=True)
targets.index.name = 'id'
targets.to_csv('train/targets.csv')
targets

,text
id,
G1T1,Poner fin a la pobreza en todas sus formas en ...
G1T2,Poner fin a la pobreza en todas sus formas en ...
G1T3,Poner fin a la pobreza en todas sus formas en ...
G1T4,Poner fin a la pobreza en todas sus formas en ...
G1T5,Poner fin a la pobreza en todas sus formas en ...
...,...
G16T14,"Promover sociedades justas, pacíficas e inclus..."
G16T15,"Promover sociedades justas, pacíficas e inclus..."
G16T16,"Promover sociedades justas, pacíficas e inclus..."


In [38]:
sdg = undp[['sdg', 'sdg_information', 'target_information']].groupby(['sdg', 'sdg_information'], as_index=False).sum()
sdg['text'] = sdg['sdg_information'] + sdg['target_information']
sdg = sdg[['sdg', 'text']]
sdg.set_index('sdg', inplace=True)
sdg.index.name = 'id'
sdg.to_csv('train/sdg.csv')
sdg

,text
id,
G1,Poner fin a la pobreza en todas sus formas en ...
G10,Reducir la desigualdad en y entre los países. ...
G11,"Lograr que las ciudades sean más inclusivas, s..."
G12,Garantizar modalidades de consumo y producción...
G13,Adoptar medidas urgentes para combatir el camb...
G14,Conservar y utilizar sosteniblemente los océan...
G15,"Gestionar sosteniblemente los bosques, luchar ..."
G16,"Promover sociedades justas, pacíficas e inclus..."
G2,"Poner fin al hambre. Para 2030, poner fin al h..."
